In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nih-chest-xrays/data")

print("Path to dataset files:", path)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Resuming download from 32995540992 bytes (12100609239 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/nih-chest-xrays/data?dataset_version_number=3 (32995540992/45096150231) bytes left.


100%|██████████| 42.0G/42.0G [06:11<00:00, 32.6MB/s]  

Extracting files...


Path to dataset files: /Users/pedrosof/.cache/kagglehub/datasets/nih-chest-xrays/data/versions/3


In [5]:
import os
import shutil
import random

# Diretório original baixado pelo kagglehub
original_dir = path

# Novo dataset reduzido
dataset_dir = "/Users/pedrosof/Documents/FIAP/Trabalhos/TIAOR_2/CardioAI--Fase4/chest_xray"
os.makedirs(dataset_dir, exist_ok=True)

splits = ["train", "val", "test"]
classes = ["NORMAL", "ANOMALY"]

for split in splits:
    for cls in classes:
        os.makedirs(os.path.join(dataset_dir, split, cls), exist_ok=True)

# Listar alguns arquivos (DICOM/JPG/PNG)
all_images = []
for root, dirs, files in os.walk(original_dir):
    for f in files:
        if f.lower().endswith(("png", "jpg", "jpeg")):
            all_images.append(os.path.join(root, f))

print("Total de imagens encontradas:", len(all_images))

# Selecionar subconjunto (ex.: 300 imagens)
SAMPLES = 300
subset = random.sample(all_images, SAMPLES)

# Divisão 70/15/15
train_split = int(0.7 * SAMPLES)
val_split = int(0.15 * SAMPLES)
test_split = SAMPLES - train_split - val_split

train_files = subset[:train_split]
val_files = subset[train_split:train_split+val_split]
test_files = subset[train_split+val_split:]

def copy_files(file_list, split_name):
    for f in file_list:
        cls = random.choice(classes)  # Como não há labels no NIH, classificamos artificialmente
        dest = os.path.join(dataset_dir, split_name, cls, os.path.basename(f))
        shutil.copy(f, dest)

copy_files(train_files, "train")
copy_files(val_files, "val")
copy_files(test_files, "test")

print("Dataset reduzido em:", dataset_dir)

Total de imagens encontradas: 112120
Dataset reduzido em: /Users/pedrosof/Documents/FIAP/Trabalhos/TIAOR_2/CardioAI--Fase4/chest_xray


In [1]:
import os # Importar os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. Redução do tamanho da imagem para economizar RAM
IMG_HEIGHT = 150 
IMG_WIDTH = 150
BATCH_SIZE = 8 # Mantido em 8

# Assumindo que dataset_dir foi definido em uma célula anterior
# Exemplo: dataset_dir = "/Users/pedrosof/Documents/FIAP/Trabalhos/TIAOR_2/CardioAI--Fase4" 

train_dir = os.path.join(dataset_dir, "train")
val_dir   = os.path.join(dataset_dir, "val")
test_dir  = os.path.join(dataset_dir, "test")

# 2. Simplificar o datagen temporariamente, ou manter o seu, com atenção ao ponto 3.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 3. Adicionar color_mode se as imagens forem P&B (Comum em Raio-X)
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(IMG_HEIGHT,IMG_WIDTH), 
    batch_size=BATCH_SIZE, 
    class_mode='binary',
    color_mode='grayscale' # Use 'grayscale' se forem P&B
)

val_generator = val_datagen.flow_from_directory(
    val_dir, 
    target_size=(IMG_HEIGHT,IMG_WIDTH), 
    batch_size=BATCH_SIZE, 
    class_mode='binary', 
    shuffle=False,
    color_mode='grayscale' # Use 'grayscale' se forem P&B
)

test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(IMG_HEIGHT,IMG_WIDTH), 
    batch_size=BATCH_SIZE, 
    class_mode='binary', 
    shuffle=False,
    color_mode='grayscale' # Use 'grayscale' se forem P&B
)

train_generator.class_indices

: 

In [ ]:
x_batch, y_batch = next(train_generator)

plt.figure(figsize=(10,8))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(x_batch[i])
    plt.title(f"Classe: {int(y_batch[i])}")
    plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
# 4) Visualizar algumas imagens pós-pré-processamento (apenas para conferência)
x_batch, y_batch = next(train_generator)

plt.figure(figsize=(10, 8))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(x_batch[i])
    plt.title(f"Classe: {int(y_batch[i])}")
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
from textwrap import dedent

print(dedent(f"""
Pipeline de Pré-processamento – CardioAI Fase 4

1. Download automático:
   - Dataset NIH Chest X-Ray baixado via kagglehub.
   - Evita dependências com Google Drive ou composições manuais.

2. Tratamento inicial:
   - Dataset NIH tem 112.000+ imagens sem classes organizadas.
   - Criado subconjunto reduzido de {SAMPLES} imagens para execução rápida no Colab.

3. Organização artificial das classes:
   - Como o dataset NIH não tem rotulagem direta, as classes NORMAL e ANOMALY são atribuídas aleatoriamente apenas para fins educacionais.
   - Simula um cenário real de classificação binária.

4. Pré-processamento:
   - Redimensionamento para 224x224 (compatível com CNNs e modelos pré-treinados).
   - Normalização em 0–1.
   - Data augmentation leve no treino (rotação, zoom, flip).
   - Conjuntos train/val/test criados na proporção 70/15/15.

5. Justificativas:
   - kagglehub simplifica e centraliza o acesso ao dataset.
   - Subset reduzido viabiliza treinamento rápido.
   - Padronização prepara o pipeline para a Parte 2 (CNN e Transfer Learning).
"""))

In [ ]:
# CardioAI - Fase 4 - Parte 2: CNN + Transfer Learning
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix

IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
train_generator = None  # substitua carregando o gerador do notebook anterior
val_generator = None
test_generator = None

def create_simple_cnn(input_shape=(224,224,3)):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# exemplo (comente se não tiver os generators carregados)
# model = create_simple_cnn()
# model.fit(train_generator, validation_data=val_generator, epochs=5)
